Some SQL Joins examples

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructField, StructType

In [3]:
# Create SparkSession
spark = SparkSession.builder\
             .master("local[1]")\
             .appName("spark-app-version-x")\
             .getOrCreate()

24/03/20 11:42:58 WARN Utils: Your hostname, sasa-1-2 resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/03/20 11:42:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/20 11:42:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Read taxi data
local_file = '/home/sasa/Downloads/Code/notebooks/datasets/parquet/'
df = spark.read.parquet(local_file)

In [5]:
# DF is like a relation table in memory. Let's see the columns
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [6]:
# Query sample, using Spark SQL
df.createOrReplaceTempView('tbl_raw_yellow_taxis')

In [7]:
# SQL Statement
spark.sql('''
          select *
          from tbl_raw_yellow_taxis
          ''').show(n=5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2023-03-01 00:06:43|  2023-03-01 00:16:43|              1|          0.0|         1|                 N|         238|          42|           2|        8.6|  1.0|    0.5|       0.

Let's create a real "dimension" table, for our RateCodeID
Standard rate
JFK
Newark
Nassau or Westchester
Negotiated fare
Group ride

In [21]:
# Add Rate Code IDs
data = [("1", "Standard rate"), ("2", "JFK"), ("3", "Newark"),("4","Nassau or Westchester "),("5","Negotiated fare"), ("6","Group ride")]

In [22]:
# Define schema, to ensure data types
schema = StructType([ \
    StructField("rate_code_ID",StringType(),True), \
    StructField("RatecodeName",StringType(),True)
  ])

In [23]:
# Create Dataframe for Rate Codes
df_rate_codes = spark.createDataFrame(data=data,schema=schema)

In [24]:
# Show rates
df_rate_codes.show()

+------------+--------------------+
|rate_code_ID|        RatecodeName|
+------------+--------------------+
|           1|       Standard rate|
|           2|                 JFK|
|           3|              Newark|
|           4|Nassau or Westche...|
|           5|     Negotiated fare|
|           6|          Group ride|
+------------+--------------------+



In [25]:
# Inner join example:
df.join(df_rate_codes, df["RatecodeID"] == df_rate_codes["rate_code_ID"], "inner").show(n=5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------+---------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|rate_code_ID|   RatecodeName|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------+---------------+
|       2| 2023-03-01 00:29:33|  2023-03-01 00:29:36|              1|         0.29|         5|      

In [26]:
# Left join example, showing those WITH NO rate code:
df.join(df_rate_codes, df["RatecodeID"] == df_rate_codes["rate_code_ID"], "left").where("RatecodeID is NULL").show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|rate_code_ID|RatecodeName|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------+------------+
|       2| 2023-03-01 00:42:33|  2023-03-01 00:49:05|           NULL|         1.38|      NULL|              N

In [30]:
# SQL Statement
spark.sql('''
          select RatecodeID, count(1)
          from tbl_raw_yellow_taxis
          group by RatecodeID
          ''').show(n=20)

+----------+--------+
|RatecodeID|count(1)|
+----------+--------+
|      NULL|  255126|
|         6|      14|
|         5|   45262|
|         1| 8849004|
|         3|   30360|
|         2|  361464|
|         4|   15839|
|        99|   48902|
+----------+--------+



In [31]:
# If user doesn't want to see NULL values
df_na_rate_codes = df.na.fill(value=0,subset=["RatecodeID"])

In [32]:
# Left join example, showing those WITH NO rate code:
df_na_rate_codes.join(df_rate_codes, df_na_rate_codes["RatecodeID"] == df_rate_codes["rate_code_id"], "left").where("RatecodeID is NULL").show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|rate_code_ID|RatecodeName|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------+------------+
+--------+--------------------+---------------------+---------------+-------------+----------+---------------

In [33]:
# Confirm counts
df_na_rate_codes.createOrReplaceTempView('tbl_na_rate_codes')

In [35]:
# SQL Statement
spark.sql('''
          select RatecodeID, count(1)
          from tbl_na_rate_codes
          group by RatecodeID
          ''').show(n=20)

+----------+--------+
|RatecodeID|count(1)|
+----------+--------+
|         0|  255126|
|         6|      14|
|         5|   45262|
|         1| 8849004|
|         3|   30360|
|         2|  361464|
|         4|   15839|
|        99|   48902|
+----------+--------+



In [36]:
# Stop the session
spark.stop()